In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import jieba_fast as jieba
from tqdm import tqdm
import time

In [2]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}  # 词汇表中单词到索引的映射
        self.idx2word = {}  # 索引到词汇表中单词的映射
        self.idx = 0  # 当前索引

    def __len__(self):
        return len(self.word2idx)  # 返回词汇表中的单词数量

    def add_word(self, word):
        if word not in self.word2idx:
            # 如果单词不在词汇表中，将其添加到词汇表，并分配一个索引
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1  # 为下一个单词增加索引值


In [3]:
class Corpus(object):
    def __init__(self):
        self.dictionary = Vocabulary()

    def preprocess(self, text):
        ads = ['本书来自www.cr173.com免费txt小说下载站', '更多更新免费电子书请关注www.cr173.com', '----〖新语丝电子文库(www.xys.org)〗', '新语丝电子文库']
        for ad in ads:
            text = text.replace(ad, '')  # 去除广告文本
        words = jieba.lcut(text) + ['<eos>']  # 使用结巴分词对文本进行分词，并添加结束符
        return words

    def build_dictionary(self, name):
        path = 'Data/' + name + '.txt'
        ads = ['本书来自www.cr173.com免费txt小说下载站', '更多更新免费电子书请关注www.cr173.com', '----〖新语丝电子文库(www.xys.org)〗', '新语丝电子文库']
        with open(path, 'r', encoding="utf-8") as f:
            for line in f.readlines():
                for ad in ads:
                    line = line.replace(ad, '')  # 去除广告文本
                words = self.preprocess(line)
                for word in words:
                    self.dictionary.add_word(word)  # 将单词添加到词汇表

    def get_data(self, name, batch_size=20):
        self.build_dictionary(name)  # 构建词汇表

        path = 'Data/' + name + '.txt'
        tokens = 0
        with open(path, 'r', encoding="utf-8") as f:
            for line in f.readlines():
                words = self.preprocess(line)
                tokens += len(words)  # 统计标记的数量

        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r', encoding="utf-8") as f:
            for line in f.readlines():
                words = self.preprocess(line)
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]  # 将单词转换为索引
                    token += 1

        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches * batch_size]
        ids = ids.view(batch_size, -1)
        return ids


In [4]:
class LSTMmodel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMmodel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # 嵌入层，用于将索引转换为向量表示
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)  # LSTM 层
        self.linear = nn.Linear(hidden_size, vocab_size)  # 线性层，用于将隐藏状态转换为词汇表大小的输出

    def forward(self, x, h):
        x = self.embedding(x)  # 输入通过嵌入层进行转换
        out, (h, c) = self.lstm(x, h)  # LSTM 层的前向传播
        out = out.reshape(out.size(0) * out.size(1), out.size(2))  # 调整输出形状
        out = self.linear(out)  # 线性层的前向传播，得到最终输出
        return out, (h, c)


In [44]:
embed_size = 128  # 嵌入向量的维度大小
hidden_size = 1024  # LSTM 隐藏层的大小
num_layers = 1  # LSTM 层的数量
num_epochs = 50  # 训练的轮数
batch_size = 50  # 每个批次的样本数量
seq_length = 100  # 输入序列的长度
learning_rate = 0.001  # 学习率
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 设备选择（CUDA 或 CPU）


In [45]:
corpus = Corpus()  # 创建 Corpus
a = time.time()
ids = corpus.get_data('雪山飞狐', batch_size)  # 获取数据
vocab_size = len(corpus.dictionary)  # 获取词汇表大小
print(time.time() - a)


1.2292730808258057


In [47]:
model = LSTMmodel(vocab_size, embed_size, hidden_size, num_layers).to(device)  # 创建 LSTM 模型
criterion = nn.CrossEntropyLoss()  # 定义交叉熵损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # 定义 Adam 优化器

In [48]:
start=time.time()
for epoch in range(num_epochs):  # 遍历每个 epoch
    hidden = (torch.zeros(num_layers, batch_size, hidden_size).to(device),  # 初始化隐藏状态
              torch.zeros(num_layers, batch_size, hidden_size).to(device))

    for i in tqdm(range(0, ids.size(1) - seq_length, seq_length)):  # 遍历数据进行训练
        inputs = ids[:, i:i+seq_length].to(device)  # 获取输入序列
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)  # 获取目标序列

        hidden = [h.detach() for h in hidden]  # 分离隐藏状态，断开计算图
        outputs, hidden = model(inputs, hidden)  # 前向传播计算输出和更新隐藏状态
        loss = criterion(outputs, targets.reshape(-1))  # 计算损失

        model.zero_grad()  # 清空梯度
        loss.backward()  # 反向传播计算梯度
        clip_grad_norm_(model.parameters(), 0.5)  # 梯度裁剪，防止梯度爆炸
        optimizer.step()  # 更新模型参数

    print('Epoch:', epoch+1,'/',num_epochs)  # 打印当前 epoch
print('Train Time:',time.time()-start)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  5.27it/s]


Epoch: 1 / 50


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  5.89it/s]


Epoch: 2 / 50


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  5.91it/s]


...

Epoch: 48 / 50


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  5.58it/s]


Epoch: 49 / 50


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  5.59it/s]

Epoch: 50 / 50
Train Time: 160.06556344032288


In [57]:
num_samples = 150  # 要生成的样本数量
article = '那陶百岁若是年轻上二十岁，刘元鹤原不是他的敌手。他向以力大招猛见长，现下年纪一老，精力究已衰退，与刘元鹤单打独斗已相形见绌'  # 初始文本
words = jieba.lcut(article)  # 对初始文本进行分词
start_id = corpus.dictionary.word2idx[words[len(words)-1]]  # 获取初始词的索引

hidden = (torch.zeros(num_layers, 1, hidden_size).to(device),  # 初始化隐藏状态
          torch.zeros(num_layers, 1, hidden_size).to(device))

prob = torch.ones(vocab_size)  # 初始化概率向量
input_word = torch.tensor([[start_id]]).to(device)  # 将初始词转换为张量，并移动到指定设备上


In [58]:
for i in range(num_samples):
    output, hidden = model(input_word, hidden)  # 生成输出和更新隐藏状态

    prob = output.exp()  # 计算概率
    word_id = torch.multinomial(prob, num_samples=1).item()  # 根据概率进行多项式抽样得到词的索引
    input_word.fill_(word_id)  # 将输入词更新为抽样得到的词的索引
    word = corpus.dictionary.idx2word[word_id]  # 获取抽样得到的词
    word = '\n' if word == '<eos>' else word  # 如果是特殊标记<eos>，则将其转换为空行
    article += word  # 将词添加到生成的文本中

print(article)  # 打印生成的样本文本


那陶百岁若是年轻上二十岁，刘元鹤原不是他的敌手。他向以力大招猛见长，现下年纪一老，精力究已衰退，与刘元鹤单打独斗已相形见绌阴车门自成在刘元鹤中茶水收起在下这一羽箭，只使得著从曹云奇手里捧了一个弯，就在雪中寻找空手的歌辞相答，难道洞穴另有入口踢到了事迹。刘元鹤哈哈大笑，道：「锦毛她到了去睡宝树。我听爹爹说他性命？」阮士中道：「小人当时你也这姓父亲伤人托人甚深，我逃去了我瞧他。」忽听也说的声音道：「那晚宝树？」苗若兰一声：「是我放人走路，好说好是好朋友的好人？」

　　刘元鹤笑道：「咱们还是是饮马川『打遍天下无敌手』，原是自尽死的？」她虽矜持想，但
